In [1]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [2]:
connections.connect("default", host="localhost", port="19530")

In [3]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)


In [4]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  


In [5]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)


alloc_timestamp unimplemented, ignore it


Status(code=0, message=)

In [6]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])


In [7]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])

In [8]:
result

[{'pk': 4,
  'random': -11.0,
  'embeddings': [0.15948623,
   0.8803514,
   0.24836688,
   0.77581424,
   0.96341354,
   0.3557045,
   0.6410173,
   0.7626325]},
 {'pk': 9,
  'random': -13.0,
  'embeddings': [0.99726987,
   0.105793424,
   0.87336004,
   0.037706286,
   0.46225262,
   0.18037347,
   0.53756803,
   0.9451666]},
 {'pk': 13,
  'random': -11.0,
  'embeddings': [0.97808176,
   0.23090602,
   0.844524,
   0.15976019,
   0.5509891,
   0.9101951,
   0.020236522,
   0.53189063]},
 {'pk': 16,
  'random': -13.0,
  'embeddings': [0.35650438,
   0.06564371,
   0.19543861,
   0.07563919,
   0.0064547653,
   0.028276186,
   0.29776326,
   0.12323559]},
 {'pk': 18,
  'random': -13.0,
  'embeddings': [0.88635874,
   0.8856445,
   0.06731797,
   0.3495585,
   0.9143664,
   0.9585458,
   0.02626725,
   0.892007]},
 {'pk': 33,
  'random': -11.0,
  'embeddings': [0.9520698,
   0.029276751,
   0.4562799,
   0.21700542,
   0.42749605,
   0.14598852,
   0.8258859,
   0.33711535]},
 {'pk': 34,

In [15]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])

In [24]:
print(result)

['["id: 349, distance: 0.23157359659671783, entity: {\'random\': -11.0}", "id: 2477, distance: 0.31499212980270386, entity: {\'random\': -11.0}", "id: 770, distance: 0.36039620637893677, entity: {\'random\': -11.0}"]', '["id: 2999, distance: 0.0, entity: {\'random\': -11.0}", "id: 2532, distance: 0.18113523721694946, entity: {\'random\': -11.0}", "id: 497, distance: 0.21876057982444763, entity: {\'random\': -11.0}"]']


In [25]:
expr = f"pk in [{entities[0][0]}, {entities[0][1]}]"
hello_milvus.delete(expr)

(insert count: 0, delete count: 2, upsert count: 0, timestamp: 444465818562723842, success count: 0, err count: 0)

In [ ]:
utility.drop_collection("hello_milvus")